# __Battle of the neighborhoods (IBM Capstone project)__

## In this notebook, I want to find out how many vegan restaurants there are in Berlin and in which neighborhoods they are clustered. 

# The steps are:
- Obtain neighborhood data by webscraping Wikipedia and constructing a data frame 
- Obtain the geographical coordinates for the neighborhoods using geocoder
- Obtain information on the most popular venues per neighborhood and obtain information on our target variable (vegan restaurant) using the Foursquare API
- Explore the neighborhoods and then cluster for our target variable 
- Select the best cluster for our new vegan restaurant! 

## 1. Install packages

In [41]:
!pip install folium
!pip install geopandas
!pip install geocoder

     |████████████████████████████████| 102kB 7.3MB/s ta 0:00:011


## 2. Import packages

In [42]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import requests
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


## 3. Alright, let's get Berlin's neighborhoods! 

In [37]:
#scrape the districts of Berlin, Germany from the Wikipedia page
url='https://en.wikipedia.org/wiki/Boroughs_and_neighborhoods_of_Berlin'
#put data in a pandas dataframe
df=pd.read_html(url, header=0)[0]
df.head()

,Borough,Population 31 March 2010,Area in km²,Density per km²,Map
0,Charlottenburg-Wilmersdorf,319628,64.72,4878,NaN
1,Friedrichshain-Kreuzberg,268225,20.16,13187,NaN
2,Lichtenberg,259881,52.29,4952,NaN
3,Marzahn-Hellersdorf,248264,61.74,4046,NaN
4,Mitte,332919,39.47,8272,NaN


In [62]:
#look at shape
df.shape
df.head()

,Borough
0,Charlottenburg-Wilmersdorf
1,Friedrichshain-Kreuzberg
2,Lichtenberg
3,Marzahn-Hellersdorf
4,Mitte


In [63]:
#drop columns we don't need
df.drop(['Population 31 March 2010','Area in km²', 'Density per km²', 'Map'], axis=1, inplace=True)
df.head()

KeyError: "['Population 31 March 2010' 'Area in km²' 'Density per km²' 'Map'] not found in axis"

## 4. Okay but we are missing the geo data, let's get that using geocoder

In [47]:
#get coordinates
import geocoder 

# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Cologne, Germany'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [49]:
# initialize new list with coordinates
c = [ get_latlng(neighborhood) for neighborhood in df["Borough"].tolist() ]

In [52]:
#make a pandas dataframe with coordinates
df_c = pd.DataFrame(c, columns=['Latitude', 'Longitude'])
df_c

,Latitude,Longitude
0,52.499620,13.323160
1,52.500000,13.450000
2,50.937420,6.947560
3,51.013100,6.900460
4,50.856095,7.111685
5,52.480770,13.435410
6,50.941670,6.955160
7,52.575450,13.349700
8,50.941670,6.955160
9,52.434850,13.241830


In [65]:
#merge two dataframes 
df['Latitude'] = df_c['Latitude']
df['Longitude'] = df_c['Longitude']

In [68]:
#check whether it worked
df.shape
df.head()

,Borough,Latitude,Longitude
0,Charlottenburg-Wilmersdorf,52.499620,13.323160
1,Friedrichshain-Kreuzberg,52.500000,13.450000
2,Lichtenberg,50.937420,6.947560
3,Marzahn-Hellersdorf,51.013100,6.900460
4,Mitte,50.856095,7.111685


In [70]:
from geopy import Nominatim
address = 'Berlin'

geolocator = Nominatim(user_agent="berlin_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Berlin are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Berlin are 52.5170365, 13.3888599.


## 5. Let's make our Berlin map!

In [72]:
# create map of Berlin using latitude and longitude values
import folium
map_b = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough in zip(df['Latitude'], df['Longitude'], df['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_b)  
    
map_b

## 6. Initialize FS API to get venue data

In [74]:
#use foursquare location API 
CLIENT_ID = 'O1ISPF1ECFGIVHATNNG12RU5NPVAMO4GN2TJ1WWLLJ3DVBYE' # your Foursquare ID
CLIENT_SECRET = 'PMEKMU3HJONKTNBRMR5COBUFQVQ4MOGJI1VFU4YAC4Z411LI' # your Foursquare Secret
VERSION = '20200703' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: O1ISPF1ECFGIVHATNNG12RU5NPVAMO4GN2TJ1WWLLJ3DVBYE
CLIENT_SECRET:PMEKMU3HJONKTNBRMR5COBUFQVQ4MOGJI1VFU4YAC4Z411LI


## 7. Okay let's check out the 100 top venues in Berlin with a radius of 5km!

In [88]:
#get top 100 venues in Berlin
limit = 100
radius = 5000

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [89]:
venues = getNearbyVenues(names=df['Borough'],latitudes=df['Latitude'],longitudes=df['Longitude'])

Charlottenburg-Wilmersdorf
Friedrichshain-Kreuzberg
Lichtenberg
Marzahn-Hellersdorf
Mitte
Neukölln
Pankow
Reinickendorf
Spandau
Steglitz-Zehlendorf
Tempelhof-Schöneberg
Treptow-Köpenick


In [90]:
print(venues.shape)
venues.head()

(521, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Charlottenburg-Wilmersdorf,52.49962,13.32316,Mövenpick Weinkeller,52.499556,13.320365,Wine Shop
1,Charlottenburg-Wilmersdorf,52.49962,13.32316,Sweet2go,52.497776,13.322638,Dessert Shop
2,Charlottenburg-Wilmersdorf,52.49962,13.32316,BERLINRODEO interior concepts GmbH,52.501163,13.325814,Furniture / Home Store
3,Charlottenburg-Wilmersdorf,52.49962,13.32316,Piccola Taormina,52.501231,13.324884,Italian Restaurant
4,Charlottenburg-Wilmersdorf,52.49962,13.32316,EDEKA,52.500454,13.322082,Supermarket


## Okay, how many unique categories are there?

In [99]:
#how many unique categories are there? 
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 151 uniques categories.


In [100]:
# print out the list of the 151 categories
venues['Venue Category'].unique()[:151]

array(['Wine Shop', 'Dessert Shop', 'Furniture / Home Store',
       'Italian Restaurant', 'Supermarket', 'Pastry Shop',
       'South American Restaurant', 'Breakfast Spot', 'Café',
       'Szechuan Restaurant', 'Argentinian Restaurant',
       'Vietnamese Restaurant', 'Middle Eastern Restaurant',
       'Japanese Restaurant', 'Gourmet Shop', 'Drugstore', 'Bar', 'Plaza',
       'Movie Theater', 'Bakery', 'Hotel', 'French Restaurant',
       'Bookstore', 'Portuguese Restaurant', 'Seafood Restaurant',
       'Indonesian Restaurant', 'Liquor Store', 'Camera Store',
       'Performing Arts Venue', 'Thai Restaurant', 'Comedy Club',
       'Tapas Restaurant', 'German Restaurant', 'Snack Place',
       'Russian Restaurant', 'Organic Grocery', 'Asian Restaurant',
       'Health & Beauty Service', 'Cocktail Bar', 'Restaurant',
       'Greek Restaurant', 'Coffee Shop', 'Indian Restaurant',
       'Optical Shop', 'Boutique', 'Brasserie', 'Sushi Restaurant',
       "Men's Store", 'Art Museum', 'P

In [118]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Charlottenburg-Wilmersdorf,100,100,100,100,100,100
Friedrichshain-Kreuzberg,35,35,35,35,35,35
Lichtenberg,75,75,75,75,75,75
Marzahn-Hellersdorf,5,5,5,5,5,5
Mitte,4,4,4,4,4,4
Neukölln,70,70,70,70,70,70
Pankow,100,100,100,100,100,100
Reinickendorf,6,6,6,6,6,6
Spandau,100,100,100,100,100,100


In [91]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,Neighborhood,ATM,Accessories Store,Airport Service,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Automotive Shop,Baby Store,Bakery,Bank,Bar,Beer Garden,Bike Shop,Bistro,Bookstore,Boutique,Brasserie,Bratwurst Joint,Breakfast Spot,Brewery,Bridge,Burger Joint,Bus Stop,Café,Camera Store,Cantonese Restaurant,Caucasian Restaurant,Cheese Shop,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Currywurst Joint,Department Store,Dessert Shop,Distillery,Dive Bar,Doner Restaurant,Drugstore,Eastern European Restaurant,Exhibit,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Go Kart Track,Gourmet Shop,Greek Restaurant,Gym / Fitness Center,Health & Beauty Service,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lebanese Restaurant,Light Rail Station,Liquor Store,Lottery Retailer,Lounge,Market,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Museum,Newsstand,Nightclub,Opera House,Optical Shop,Organic Grocery,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pharmacy,Photography Studio,Platform,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Restaurant,Rhenisch Restaurant,Road,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Schnitzel Restaurant,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soccer Field,Soup Place,South American Restaurant,Speakeasy,Sporting Goods Shop,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Train Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Charlottenburg-Wilmersdorf,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,Charlottenburg-Wilmersdorf,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Charlottenburg-Wilmersdorf,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Charlottenburg-Wilmersdorf,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Charlottenburg-Wilmersdorf,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [92]:
#how big is the new dataframe?
onehot.shape

(521, 152)

In [93]:
#group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,ATM,Accessories Store,Airport Service,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Automotive Shop,Baby Store,Bakery,Bank,Bar,Beer Garden,Bike Shop,Bistro,Bookstore,Boutique,Brasserie,Bratwurst Joint,Breakfast Spot,Brewery,Bridge,Burger Joint,Bus Stop,Café,Camera Store,Cantonese Restaurant,Caucasian Restaurant,Cheese Shop,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Currywurst Joint,Department Store,Dessert Shop,Distillery,Dive Bar,Doner Restaurant,Drugstore,Eastern European Restaurant,Exhibit,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Go Kart Track,Gourmet Shop,Greek Restaurant,Gym / Fitness Center,Health & Beauty Service,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lebanese Restaurant,Light Rail Station,Liquor Store,Lottery Retailer,Lounge,Market,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Museum,Newsstand,Nightclub,Opera House,Optical Shop,Organic Grocery,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pharmacy,Photography Studio,Platform,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Restaurant,Rhenisch Restaurant,Road,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Schnitzel Restaurant,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soccer Field,Soup Place,South American Restaurant,Speakeasy,Sporting Goods Shop,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Train Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Charlottenburg-Wilmersdorf,0.00,0.000000,0.00,0.01,0.000000,0.010000,0.01,0.0,0.000000,0.020000,0.00,0.040000,0.000000,0.000000,0.000000,0.010000,0.030000,0.01,0.00,0.030000,0.000000,0.000000,0.010000,0.000000,0.070000,0.010000,0.00,0.000000,0.000000,0.000000,0.00,0.010000,0.030000,0.010000,0.01,0.00,0.000000,0.00,0.00,0.00,0.00,0.040000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.02,0.000000,0.000000,0.020000,0.0,0.000000,0.000000,0.030000,0.000000,0.000000,0.010000,0.01,0.000000,0.01,0.000000,0.080000,0.000000,0.030000,0.000000,0.01,0.000000,0.120000,0.01,0.000000,0.00,0.000000,0.000000,0.000000,0.010000,0.0,0.000000,0.000000,0.01,0.000000,0.010000,0.00,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.01,0.01,0.00,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000,0.010000,0.000000,0.000000,0.000000,0.01,0.000000,0.01,0.000000,0.00,0.000000,0.01,0.0,0.00,0.010000,0.000000,0.000000,0.01,0.000000,0.000000,0.00,0.00,0.000000,0.020000,0.010000,0.01,0.010000,0.01,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.010000,0.000000,0.010000,0.020000,0.000000,0.01
1,Friedrichshain-Kreuzberg,0.00,0.000000,0.00,0.00,0.028571,0.000000,0.00,0.0,0.000000,0.028571,0.00,0.000000,0.028571,0.028571,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.028571,0.000000,0.000000,0.057143,0.000000,0.00,0.028571,0.000000,0.000000,0.00,0.000000,0.000000,0.028571,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.028571,0.000000,0.000000,0.000000,0.000000,0.028571,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.028571,0.000000,0.000000,0.000000,0.00,0.028571,0.00,0.028571,0.028571,0.085714,0.028571,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0

## 8. Okay, let's check for our target variable, vegan restaurants!

In [101]:
#now let us check for how many vegan restaurants we have!
len(grouped[grouped['Vegetarian / Vegan Restaurant'] > 0])

5

In [104]:
veg = grouped[['Neighborhood', 'Vegetarian / Vegan Restaurant']]
veg

,Neighborhood,Vegetarian / Vegan Restaurant
0,Charlottenburg-Wilmersdorf,0.010000
1,Friedrichshain-Kreuzberg,0.000000
2,Lichtenberg,0.013333
3,Marzahn-Hellersdorf,0.000000
4,Mitte,0.000000
5,Neukölln,0.028571
6,Pankow,0.010000
7,Reinickendorf,0.000000
8,Spandau,0.010000
9,Steglitz-Zehlendorf,0.000000


### Now we see that there are only vegan restaurants in Charlottenburg-Wilmersdorf, Lichtenberg, Neukölln, Pankow and Spandau. Cool!

## Let's now check the 5 top venues per neighborhood! Maybe vegan will be in there.

In [94]:
#print top 5 venues per neighborhood group
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Charlottenburg-Wilmersdorf----
                venue  freq
0  Italian Restaurant  0.12
1               Hotel  0.08
2                Café  0.07
3        Dessert Shop  0.04
4                 Bar  0.04


----Friedrichshain-Kreuzberg----
            venue  freq
0  Ice Cream Shop  0.09
1       Nightclub  0.06
2       Rock Club  0.06
3            Café  0.06
4      Distillery  0.03


----Lichtenberg----
                venue  freq
0  Italian Restaurant  0.07
1                Café  0.05
2          Restaurant  0.04
3               Hotel  0.04
4         Salad Place  0.03


----Marzahn-Hellersdorf----
           venue  freq
0  Metro Station   0.2
1          Plaza   0.2
2  Shopping Mall   0.2
3      Drugstore   0.2
4     Restaurant   0.2


----Mitte----
                venue  freq
0               Hotel  0.50
1  Italian Restaurant  0.25
2    Greek Restaurant  0.25
3                 ATM  0.00
4        Perfume Shop  0.00


----Neukölln----
                       venue  freq
0                     

### We can see that vegan is not in there, but Italian is in almost every neighborhood! Maybe a vegan Italian restaurant would be perfect!

In [95]:
#sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [96]:
#create pandas dataframe with top 10 venues per neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Charlottenburg-Wilmersdorf,Italian Restaurant,Hotel,Café,Bar,Dessert Shop,German Restaurant,Boutique,Breakfast Spot,Indian Restaurant,Cocktail Bar
1,Friedrichshain-Kreuzberg,Ice Cream Shop,Rock Club,Nightclub,Café,Hostel,Lounge,Park,Schnitzel Restaurant,Middle Eastern Restaurant,Caucasian Restaurant
2,Lichtenberg,Italian Restaurant,Café,Hotel,Restaurant,Salad Place,Bakery,Supermarket,Sushi Restaurant,Food & Drink Shop,Pedestrian Plaza
3,Marzahn-Hellersdorf,Restaurant,Plaza,Metro Station,Shopping Mall,Drugstore,Department Store,Dessert Shop,Distillery,Dive Bar,Doner Restaurant
4,Mitte,Hotel,Italian Restaurant,Greek Restaurant,Currywurst Joint,Eastern European Restaurant,Fountain,Food Truck,Food & Drink Shop,Fast Food Restaurant,Farmers Market


## 9. Cluster for similar venues!

In [133]:
#run k-means to cluster by 5
# set number of clusters
kclusters = 10

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[1:10]

array([8, 3, 6, 4, 0, 3, 2, 3, 5], dtype=int32)

In [134]:
#let's add some cluster labels to our dataframe!
b_veg = grouped
b_veg["Cluster Labels"] = kmeans.labels_
b_veg.head()

,Neighborhood,ATM,Accessories Store,Airport Service,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Automotive Shop,Baby Store,Bakery,Bank,Bar,Beer Garden,Bike Shop,Bistro,Bookstore,Boutique,Brasserie,Bratwurst Joint,Breakfast Spot,Brewery,Bridge,Burger Joint,Bus Stop,Café,Camera Store,Cantonese Restaurant,Caucasian Restaurant,Cheese Shop,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Currywurst Joint,Department Store,Dessert Shop,Distillery,Dive Bar,Doner Restaurant,Drugstore,Eastern European Restaurant,Exhibit,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Go Kart Track,Gourmet Shop,Greek Restaurant,Gym / Fitness Center,Health & Beauty Service,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lebanese Restaurant,Light Rail Station,Liquor Store,Lottery Retailer,Lounge,Market,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Museum,Newsstand,Nightclub,Opera House,Optical Shop,Organic Grocery,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pharmacy,Photography Studio,Platform,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Restaurant,Rhenisch Restaurant,Road,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Schnitzel Restaurant,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soccer Field,Soup Place,South American Restaurant,Speakeasy,Sporting Goods Shop,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Train Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Cluster Labels,Latitude,Longitude
0,Charlottenburg-Wilmersdorf,0.0,0.000000,0.0,0.01,0.000000,0.010000,0.01,0.0,0.0,0.020000,0.0,0.040000,0.000000,0.000000,0.0,0.010000,0.030000,0.01,0.0,0.030000,0.0,0.000000,0.01,0.0,0.070000,0.010000,0.0,0.000000,0.000000,0.000000,0.0,0.010000,0.03,0.010000,0.01,0.0,0.000000,0.0,0.0,0.0,0.0,0.040000,0.000000,0.0,0.0,0.010000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.02,0.0,0.000000,0.020000,0.0,0.000000,0.000000,0.030000,0.000000,0.0,0.010000,0.01,0.000000,0.01,0.000000,0.080000,0.000000,0.030000,0.0,0.01,0.000000,0.120000,0.01,0.0,0.0,0.000000,0.0,0.0,0.010000,0.0,0.000000,0.000000,0.01,0.0,0.010000,0.0,0.0,0.01,0.000000,0.0,0.000000,0.0,0.010000,0.01,0.000000,0.000000,0.000000,0.01,0.000000,0.01,0.01,0.0,0.000000,0.0,0.010000,0.000000,0.000000,0.010000,0.0,0.01,0.000000,0.000000,0.000000,0.01,0.000000,0.01,0.0,0.0,0.000000,0.01,0.0,0.0,0.01,0.0,0.000000,0.01,0.000000,0.000000,0.0,0.0,0.0,0.020000,0.010000,0.01,0.010000,0.01,0.000000,0.0,0.0,0.000000,0.0,0.0,0.010000,0.0,0.010000,0.020000,0.000000,0.01,9,52.499620,13.323160
1,Friedrichshain-Kreuzberg,0.0,0.000000,0.0,0.00,0.028571,0.000000,0.00,0.0,0.0,0.028571,0.0,0.000000,0.028571,0.028571,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.028571,0.00,0.0,0.057143,0.000000,0.0,0.028571,0.000000,0.000000,0.0,0.000000,0.00,0.028571,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.028571,0.0,0.0,0.000000,0.0,0.028571,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.028571,0.000000,0.0,0.000000,0.00,0.028571,0.00,0.028571,0.028571,0.085714,0.028571,0.0,0.00,0.000000,0.000000,0.00,0.0,0.0,0.028571,0.0,0.0,0.000000,0.0,0.028571,0.000000,0.00,0.0,0.028571,0.0,0.0,0.00,0.028571,0.0,0.057143,0.0,0.000000,0.00,0.000000,0.000000,0.028571,0.00,0.000000,0.00,0.00,0.0,0.000000,0.0

In [135]:
#and add geo data
b_veg['Latitude'] = df_c['Latitude']
b_veg['Longitude'] = df_c['Longitude']
b_veg.head()

,Neighborhood,ATM,Accessories Store,Airport Service,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Automotive Shop,Baby Store,Bakery,Bank,Bar,Beer Garden,Bike Shop,Bistro,Bookstore,Boutique,Brasserie,Bratwurst Joint,Breakfast Spot,Brewery,Bridge,Burger Joint,Bus Stop,Café,Camera Store,Cantonese Restaurant,Caucasian Restaurant,Cheese Shop,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Currywurst Joint,Department Store,Dessert Shop,Distillery,Dive Bar,Doner Restaurant,Drugstore,Eastern European Restaurant,Exhibit,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Go Kart Track,Gourmet Shop,Greek Restaurant,Gym / Fitness Center,Health & Beauty Service,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lebanese Restaurant,Light Rail Station,Liquor Store,Lottery Retailer,Lounge,Market,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Museum,Newsstand,Nightclub,Opera House,Optical Shop,Organic Grocery,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pharmacy,Photography Studio,Platform,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Restaurant,Rhenisch Restaurant,Road,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Schnitzel Restaurant,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soccer Field,Soup Place,South American Restaurant,Speakeasy,Sporting Goods Shop,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Train Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Cluster Labels,Latitude,Longitude
0,Charlottenburg-Wilmersdorf,0.0,0.000000,0.0,0.01,0.000000,0.010000,0.01,0.0,0.0,0.020000,0.0,0.040000,0.000000,0.000000,0.0,0.010000,0.030000,0.01,0.0,0.030000,0.0,0.000000,0.01,0.0,0.070000,0.010000,0.0,0.000000,0.000000,0.000000,0.0,0.010000,0.03,0.010000,0.01,0.0,0.000000,0.0,0.0,0.0,0.0,0.040000,0.000000,0.0,0.0,0.010000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.02,0.0,0.000000,0.020000,0.0,0.000000,0.000000,0.030000,0.000000,0.0,0.010000,0.01,0.000000,0.01,0.000000,0.080000,0.000000,0.030000,0.0,0.01,0.000000,0.120000,0.01,0.0,0.0,0.000000,0.0,0.0,0.010000,0.0,0.000000,0.000000,0.01,0.0,0.010000,0.0,0.0,0.01,0.000000,0.0,0.000000,0.0,0.010000,0.01,0.000000,0.000000,0.000000,0.01,0.000000,0.01,0.01,0.0,0.000000,0.0,0.010000,0.000000,0.000000,0.010000,0.0,0.01,0.000000,0.000000,0.000000,0.01,0.000000,0.01,0.0,0.0,0.000000,0.01,0.0,0.0,0.01,0.0,0.000000,0.01,0.000000,0.000000,0.0,0.0,0.0,0.020000,0.010000,0.01,0.010000,0.01,0.000000,0.0,0.0,0.000000,0.0,0.0,0.010000,0.0,0.010000,0.020000,0.000000,0.01,9,52.499620,13.323160
1,Friedrichshain-Kreuzberg,0.0,0.000000,0.0,0.00,0.028571,0.000000,0.00,0.0,0.0,0.028571,0.0,0.000000,0.028571,0.028571,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.028571,0.00,0.0,0.057143,0.000000,0.0,0.028571,0.000000,0.000000,0.0,0.000000,0.00,0.028571,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.028571,0.0,0.0,0.000000,0.0,0.028571,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.028571,0.000000,0.0,0.000000,0.00,0.028571,0.00,0.028571,0.028571,0.085714,0.028571,0.0,0.00,0.000000,0.000000,0.00,0.0,0.0,0.028571,0.0,0.0,0.000000,0.0,0.028571,0.000000,0.00,0.0,0.028571,0.0,0.0,0.00,0.028571,0.0,0.057143,0.0,0.000000,0.00,0.000000,0.000000,0.028571,0.00,0.000000,0.00,0.00,0.0,0.000000,0.0

### Let's visualize! :)

In [136]:
# create our color coded map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(b_veg['Latitude'], b_veg['Longitude'], b_veg['Neighborhood'], b_veg['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Pretty cool! Now we can examine the clusters further.

In [137]:
#Cluster 1
#This seems to have a vegan restaurants! 
b_veg.loc[b_veg['Cluster Labels'] == 0, b_veg.columns[[1] + list(range(5, b_veg.shape[1]))]]

,ATM,Art Gallery,Art Museum,Asian Restaurant,Automotive Shop,Baby Store,Bakery,Bank,Bar,Beer Garden,Bike Shop,Bistro,Bookstore,Boutique,Brasserie,Bratwurst Joint,Breakfast Spot,Brewery,Bridge,Burger Joint,Bus Stop,Café,Camera Store,Cantonese Restaurant,Caucasian Restaurant,Cheese Shop,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Currywurst Joint,Department Store,Dessert Shop,Distillery,Dive Bar,Doner Restaurant,Drugstore,Eastern European Restaurant,Exhibit,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Go Kart Track,Gourmet Shop,Greek Restaurant,Gym / Fitness Center,Health & Beauty Service,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lebanese Restaurant,Light Rail Station,Liquor Store,Lottery Retailer,Lounge,Market,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Museum,Newsstand,Nightclub,Opera House,Optical Shop,Organic Grocery,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pharmacy,Photography Studio,Platform,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Restaurant,Rhenisch Restaurant,Road,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Schnitzel Restaurant,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soccer Field,Soup Place,South American Restaurant,Speakeasy,Sporting Goods Shop,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Train Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Cluster Labels,Latitude,Longitude
5,0.0,0.014286,0.0,0.0,0.0,0.0,0.014286,0.0,0.114286,0.014286,0.0,0.042857,0.014286,0.0,0.0,0.0,0.014286,0.014286,0.0,0.014286,0.0,0.071429,0.0,0.0,0.0,0.014286,0.0,0.0,0.0,0.042857,0.042857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028571,0.0,0.0,0.0,0.0,0.0,0.0,0.014286,0.0,0.0,0.0,0.014286,0.0,0.0,0.0,0.014286,0.0,0.014286,0.0,0.014286,0.0,0.0,0.014286,0.0,0.0,0.0,0.014286,0.0,0.014286,0.0,0.0,0.042857,0.0,0.014286,0.0,0.0,0.014286,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.042857,0.0,0.0,0.014286,0.0,0.0,0.028571,0.014286,0.0,0.014286,0.0,0.0,0.0,0.014286,0.0,0.0,0.0,0.0,0.0,0.0,0.014286,0.0,0.014286,0.0,0.028571,0.014286,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.014286,0.028571,0.014286,0.0,0.014286,0.0,0.0,0.014286,0.0,0.0,0.0,0.0,0.0,0.014286,0.028571,0.014286,0.014286,0.0,0,52.48077,13.43541


In [138]:
#Cluster 2
#This seems to have 0 egan restaurants! 
b_veg.loc[b_veg['Cluster Labels'] == 1, b_veg.columns[[1] + list(range(5, b_veg.shape[1]))]]

,ATM,Art Gallery,Art Museum,Asian Restaurant,Automotive Shop,Baby Store,Bakery,Bank,Bar,Beer Garden,Bike Shop,Bistro,Bookstore,Boutique,Brasserie,Bratwurst Joint,Breakfast Spot,Brewery,Bridge,Burger Joint,Bus Stop,Café,Camera Store,Cantonese Restaurant,Caucasian Restaurant,Cheese Shop,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Currywurst Joint,Department Store,Dessert Shop,Distillery,Dive Bar,Doner Restaurant,Drugstore,Eastern European Restaurant,Exhibit,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Go Kart Track,Gourmet Shop,Greek Restaurant,Gym / Fitness Center,Health & Beauty Service,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lebanese Restaurant,Light Rail Station,Liquor Store,Lottery Retailer,Lounge,Market,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Museum,Newsstand,Nightclub,Opera House,Optical Shop,Organic Grocery,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pharmacy,Photography Studio,Platform,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Restaurant,Rhenisch Restaurant,Road,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Schnitzel Restaurant,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soccer Field,Soup Place,South American Restaurant,Speakeasy,Sporting Goods Shop,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Train Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Cluster Labels,Latitude,Longitude
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,50.97275,6.837119


In [139]:
#Cluster 3
#This seems to have 0 vegan restaurants! 
b_veg.loc[b_veg['Cluster Labels'] == 2, b_veg.columns[[1] + list(range(5, b_veg.shape[1]))]]

,ATM,Art Gallery,Art Museum,Asian Restaurant,Automotive Shop,Baby Store,Bakery,Bank,Bar,Beer Garden,Bike Shop,Bistro,Bookstore,Boutique,Brasserie,Bratwurst Joint,Breakfast Spot,Brewery,Bridge,Burger Joint,Bus Stop,Café,Camera Store,Cantonese Restaurant,Caucasian Restaurant,Cheese Shop,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Currywurst Joint,Department Store,Dessert Shop,Distillery,Dive Bar,Doner Restaurant,Drugstore,Eastern European Restaurant,Exhibit,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Go Kart Track,Gourmet Shop,Greek Restaurant,Gym / Fitness Center,Health & Beauty Service,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lebanese Restaurant,Light Rail Station,Liquor Store,Lottery Retailer,Lounge,Market,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Museum,Newsstand,Nightclub,Opera House,Optical Shop,Organic Grocery,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pharmacy,Photography Studio,Platform,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Restaurant,Rhenisch Restaurant,Road,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Schnitzel Restaurant,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soccer Field,Soup Place,South American Restaurant,Speakeasy,Sporting Goods Shop,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Train Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Cluster Labels,Latitude,Longitude
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,52.57545,13.3497


In [140]:
#Cluster 4
#This seems to have vegan restaurants! 
b_veg.loc[b_veg['Cluster Labels'] == 3, b_veg.columns[[1] + list(range(5, b_veg.shape[1]))]]

,ATM,Art Gallery,Art Museum,Asian Restaurant,Automotive Shop,Baby Store,Bakery,Bank,Bar,Beer Garden,Bike Shop,Bistro,Bookstore,Boutique,Brasserie,Bratwurst Joint,Breakfast Spot,Brewery,Bridge,Burger Joint,Bus Stop,Café,Camera Store,Cantonese Restaurant,Caucasian Restaurant,Cheese Shop,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Currywurst Joint,Department Store,Dessert Shop,Distillery,Dive Bar,Doner Restaurant,Drugstore,Eastern European Restaurant,Exhibit,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Go Kart Track,Gourmet Shop,Greek Restaurant,Gym / Fitness Center,Health & Beauty Service,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lebanese Restaurant,Light Rail Station,Liquor Store,Lottery Retailer,Lounge,Market,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Museum,Newsstand,Nightclub,Opera House,Optical Shop,Organic Grocery,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pharmacy,Photography Studio,Platform,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Restaurant,Rhenisch Restaurant,Road,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Schnitzel Restaurant,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soccer Field,Soup Place,South American Restaurant,Speakeasy,Sporting Goods Shop,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Train Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Cluster Labels,Latitude,Longitude
2,0.00,0.0,0.013333,0.00,0.0,0.0,0.026667,0.00,0.013333,0.0,0.0,0.0,0.026667,0.013333,0.0,0.00,0.013333,0.00,0.0,0.0,0.0,0.053333,0.013333,0.00,0.0,0.013333,0.013333,0.00,0.013333,0.00,0.013333,0.00,0.00,0.013333,0.00,0.00,0.00,0.00,0.013333,0.0,0.0,0.0,0.013333,0.0,0.00,0.0,0.013333,0.026667,0.013333,0.00,0.0,0.0,0.013333,0.013333,0.0,0.013333,0.013333,0.013333,0.013333,0.0,0.013333,0.0,0.013333,0.0,0.013333,0.04,0.013333,0.0,0.0,0.0,0.013333,0.066667,0.00,0.0,0.00,0.0,0.0,0.0,0.013333,0.0,0.0,0.013333,0.0,0.0,0.013333,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.013333,0.0,0.013333,0.013333,0.0,0.0,0.026667,0.00,0.00,0.00,0.013333,0.0,0.013333,0.013333,0.0,0.0,0.00,0.04,0.013333,0.013333,0.0,0.0,0.026667,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.013333,0.00,0.00,0.0,0.026667,0.026667,0.0,0.013333,0.0,0.013333,0.0,0.00,0.013333,0.0,0.00,0.026667,0.00,0.013333,0.013333,0.013333,0.0,3,50.93742,6.94756
6,0.01,0.0,0.030000,0.02,0.0,0.0,0.060000,0.01,0.000000,0.0,0.0,0.0,0.010000,0.000000,0.0,0.01,0.000000,0.03,0.0,0.0,0.0,0.040000,0.000000,0.01,0.0,0.000000,0.010000,0.01,0.010000,0.01,0.030000,0.01,0.01,0.000000,0.01,0.02,0.01,0.01,0.000000,0.0,0.0,0.0,0.010000,0.0,0.01,0.0,0.010000,0.000000,0.000000,0.01,0.0,0.0,0.010000,0.010000,0.0,0.000000,0.000000,0.020000,0.050000,0.0,0.020000,0.0,0.010000,0.0,0.000000,0.07,0.010000,0.0,0.0,0.0,0.010000,0.070000,0.01,0.0,0.01,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.01,0.02,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.010000,0.01,0.01,0.01,0.000000,0.0,0.030000,0.010000,0.0,0.0,0.01,0.01,0.010000,0.000000,0.0,0.0,0.010000,0.0,0.0,0.01,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.020000,0.01,0.01,0.0,0.010000,0.000000,0.0,0.000000,0.0,0.010000,0.0,0.01,0.010000,0.0,0.01,0.000000,0.01,0.010000,0.000000,0.000000,0.0,3,50.94167,6.95516
8,0.01,0.0,0.030000,0.02,0.0,0.0,0.

In [141]:
#Cluster 5
#This seems to have 0 vegan restaurants! 
b_veg.loc[b_veg['Cluster Labels'] == 4, b_veg.columns[[1] + list(range(5, b_veg.shape[1]))]]

,ATM,Art Gallery,Art Museum,Asian Restaurant,Automotive Shop,Baby Store,Bakery,Bank,Bar,Beer Garden,Bike Shop,Bistro,Bookstore,Boutique,Brasserie,Bratwurst Joint,Breakfast Spot,Brewery,Bridge,Burger Joint,Bus Stop,Café,Camera Store,Cantonese Restaurant,Caucasian Restaurant,Cheese Shop,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Currywurst Joint,Department Store,Dessert Shop,Distillery,Dive Bar,Doner Restaurant,Drugstore,Eastern European Restaurant,Exhibit,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Go Kart Track,Gourmet Shop,Greek Restaurant,Gym / Fitness Center,Health & Beauty Service,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lebanese Restaurant,Light Rail Station,Liquor Store,Lottery Retailer,Lounge,Market,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Museum,Newsstand,Nightclub,Opera House,Optical Shop,Organic Grocery,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pharmacy,Photography Studio,Platform,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Restaurant,Rhenisch Restaurant,Road,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Schnitzel Restaurant,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soccer Field,Soup Place,South American Restaurant,Speakeasy,Sporting Goods Shop,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Train Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Cluster Labels,Latitude,Longitude
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,50.856095,7.111685


In [142]:
#Cluster 6
#This seems to have 0 vegan restaurants! 
b_veg.loc[b_veg['Cluster Labels'] == 5, b_veg.columns[[1] + list(range(5, b_veg.shape[1]))]]

,ATM,Art Gallery,Art Museum,Asian Restaurant,Automotive Shop,Baby Store,Bakery,Bank,Bar,Beer Garden,Bike Shop,Bistro,Bookstore,Boutique,Brasserie,Bratwurst Joint,Breakfast Spot,Brewery,Bridge,Burger Joint,Bus Stop,Café,Camera Store,Cantonese Restaurant,Caucasian Restaurant,Cheese Shop,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Currywurst Joint,Department Store,Dessert Shop,Distillery,Dive Bar,Doner Restaurant,Drugstore,Eastern European Restaurant,Exhibit,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Go Kart Track,Gourmet Shop,Greek Restaurant,Gym / Fitness Center,Health & Beauty Service,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lebanese Restaurant,Light Rail Station,Liquor Store,Lottery Retailer,Lounge,Market,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Museum,Newsstand,Nightclub,Opera House,Optical Shop,Organic Grocery,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pharmacy,Photography Studio,Platform,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Restaurant,Rhenisch Restaurant,Road,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Schnitzel Restaurant,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soccer Field,Soup Place,South American Restaurant,Speakeasy,Sporting Goods Shop,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Train Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Cluster Labels,Latitude,Longitude
9,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,52.43485,13.24183


In [144]:
#Cluster 7
#This seems to have 0 vegan restaurants! 
b_veg.loc[b_veg['Cluster Labels'] == 6, b_veg.columns[[1] + list(range(5, b_veg.shape[1]))]]

,ATM,Art Gallery,Art Museum,Asian Restaurant,Automotive Shop,Baby Store,Bakery,Bank,Bar,Beer Garden,Bike Shop,Bistro,Bookstore,Boutique,Brasserie,Bratwurst Joint,Breakfast Spot,Brewery,Bridge,Burger Joint,Bus Stop,Café,Camera Store,Cantonese Restaurant,Caucasian Restaurant,Cheese Shop,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Currywurst Joint,Department Store,Dessert Shop,Distillery,Dive Bar,Doner Restaurant,Drugstore,Eastern European Restaurant,Exhibit,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Go Kart Track,Gourmet Shop,Greek Restaurant,Gym / Fitness Center,Health & Beauty Service,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lebanese Restaurant,Light Rail Station,Liquor Store,Lottery Retailer,Lounge,Market,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Museum,Newsstand,Nightclub,Opera House,Optical Shop,Organic Grocery,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pharmacy,Photography Studio,Platform,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Restaurant,Rhenisch Restaurant,Road,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Schnitzel Restaurant,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soccer Field,Soup Place,South American Restaurant,Speakeasy,Sporting Goods Shop,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Train Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Cluster Labels,Latitude,Longitude
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,51.0131,6.90046


In [145]:
#Cluster 8
#This seems to have 0 vegan restaurants! 
b_veg.loc[b_veg['Cluster Labels'] == 7, b_veg.columns[[1] + list(range(5, b_veg.shape[1]))]]

,ATM,Art Gallery,Art Museum,Asian Restaurant,Automotive Shop,Baby Store,Bakery,Bank,Bar,Beer Garden,Bike Shop,Bistro,Bookstore,Boutique,Brasserie,Bratwurst Joint,Breakfast Spot,Brewery,Bridge,Burger Joint,Bus Stop,Café,Camera Store,Cantonese Restaurant,Caucasian Restaurant,Cheese Shop,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Currywurst Joint,Department Store,Dessert Shop,Distillery,Dive Bar,Doner Restaurant,Drugstore,Eastern European Restaurant,Exhibit,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Go Kart Track,Gourmet Shop,Greek Restaurant,Gym / Fitness Center,Health & Beauty Service,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lebanese Restaurant,Light Rail Station,Liquor Store,Lottery Retailer,Lounge,Market,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Museum,Newsstand,Nightclub,Opera House,Optical Shop,Organic Grocery,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pharmacy,Photography Studio,Platform,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Restaurant,Rhenisch Restaurant,Road,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Schnitzel Restaurant,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soccer Field,Soup Place,South American Restaurant,Speakeasy,Sporting Goods Shop,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Train Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Cluster Labels,Latitude,Longitude
10,0.0,0.0,0.0,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.052632,0.052632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.052632,0.0,0.052632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.052632,0.0,0.0,0.052632,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,0.0,0.052632,0.0,7,52.477706,13.358674


In [146]:
#Cluster 9
#This seems to have 0 vegan restaurants! 
b_veg.loc[b_veg['Cluster Labels'] == 8, b_veg.columns[[1] + list(range(5, b_veg.shape[1]))]]

,ATM,Art Gallery,Art Museum,Asian Restaurant,Automotive Shop,Baby Store,Bakery,Bank,Bar,Beer Garden,Bike Shop,Bistro,Bookstore,Boutique,Brasserie,Bratwurst Joint,Breakfast Spot,Brewery,Bridge,Burger Joint,Bus Stop,Café,Camera Store,Cantonese Restaurant,Caucasian Restaurant,Cheese Shop,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Currywurst Joint,Department Store,Dessert Shop,Distillery,Dive Bar,Doner Restaurant,Drugstore,Eastern European Restaurant,Exhibit,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Go Kart Track,Gourmet Shop,Greek Restaurant,Gym / Fitness Center,Health & Beauty Service,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lebanese Restaurant,Light Rail Station,Liquor Store,Lottery Retailer,Lounge,Market,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Museum,Newsstand,Nightclub,Opera House,Optical Shop,Organic Grocery,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pharmacy,Photography Studio,Platform,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Restaurant,Rhenisch Restaurant,Road,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Schnitzel Restaurant,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soccer Field,Soup Place,South American Restaurant,Speakeasy,Sporting Goods Shop,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Train Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Cluster Labels,Latitude,Longitude
1,0.0,0.028571,0.0,0.0,0.0,0.0,0.028571,0.0,0.0,0.028571,0.028571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028571,0.0,0.0,0.057143,0.0,0.0,0.028571,0.0,0.0,0.0,0.0,0.0,0.028571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028571,0.0,0.0,0.0,0.0,0.028571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028571,0.0,0.0,0.0,0.0,0.028571,0.0,0.028571,0.028571,0.085714,0.028571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028571,0.0,0.0,0.0,0.0,0.028571,0.0,0.0,0.0,0.028571,0.0,0.0,0.0,0.028571,0.0,0.057143,0.0,0.0,0.0,0.0,0.0,0.028571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028571,0.0,0.028571,0.028571,0.0,0.0,0.0,0.0,0.057143,0.0,0.0,0.0,0.0,0.0,0.028571,0.0,0.0,0.0,0.0,0.0,0.028571,0.0,0.028571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028571,0.0,0.0,8,52.5,13.45


In [147]:
#Cluster 10
#This seems to have a vegan restaurants! 
b_veg.loc[b_veg['Cluster Labels'] == 9, b_veg.columns[[1] + list(range(5, b_veg.shape[1]))]]

,ATM,Art Gallery,Art Museum,Asian Restaurant,Automotive Shop,Baby Store,Bakery,Bank,Bar,Beer Garden,Bike Shop,Bistro,Bookstore,Boutique,Brasserie,Bratwurst Joint,Breakfast Spot,Brewery,Bridge,Burger Joint,Bus Stop,Café,Camera Store,Cantonese Restaurant,Caucasian Restaurant,Cheese Shop,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Currywurst Joint,Department Store,Dessert Shop,Distillery,Dive Bar,Doner Restaurant,Drugstore,Eastern European Restaurant,Exhibit,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Go Kart Track,Gourmet Shop,Greek Restaurant,Gym / Fitness Center,Health & Beauty Service,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lebanese Restaurant,Light Rail Station,Liquor Store,Lottery Retailer,Lounge,Market,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Museum,Newsstand,Nightclub,Opera House,Optical Shop,Organic Grocery,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pharmacy,Photography Studio,Platform,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Restaurant,Rhenisch Restaurant,Road,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Schnitzel Restaurant,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soccer Field,Soup Place,South American Restaurant,Speakeasy,Sporting Goods Shop,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Train Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Cluster Labels,Latitude,Longitude
0,0.0,0.0,0.01,0.01,0.0,0.0,0.02,0.0,0.04,0.0,0.0,0.0,0.01,0.03,0.01,0.0,0.03,0.0,0.0,0.01,0.0,0.07,0.01,0.0,0.0,0.0,0.0,0.0,0.01,0.03,0.01,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.04,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.02,0.0,0.0,0.0,0.03,0.0,0.0,0.01,0.01,0.0,0.01,0.0,0.08,0.0,0.03,0.0,0.01,0.0,0.12,0.01,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.01,0.0,0.01,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.01,0.01,0.0,0.0,0.0,0.01,0.0,0.01,0.01,0.0,0.0,0.0,0.01,0.0,0.0,0.01,0.0,0.01,0.0,0.0,0.0,0.01,0.0,0.01,0.0,0.0,0.0,0.01,0.0,0.0,0.01,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.02,0.01,0.01,0.01,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.01,0.02,0.0,0.01,9,52.49962,13.32316


## 10. Observations

Okay, so from our analysis we can see that there are not a lot of vegan or vegetarian restaurants in Berlin - that is surprising! We only found 5 and they are pretty spread out. We also saw from our general observations, that the most popular cuisine seems to be Italian. 
Our observations tell us that areas of great opportunity are clusters 2-5, as there is zero competition there. Cluster 1 should be avoided with 5/5 vegan restaurants in these neighborhoods. 
Furthermore, if aiming for the best revenue, the vegan restaurant should be Italian themed, as this seems to be the most popular cuisine throughout all boroughs. 